In [ ]:
pip install -q yahoo_fin

In [2]:
!mkdir -p data results logs


In [3]:
from google.colab import drive
drive.mount('/content/drive')
!cp  /content/drive/MyDrive/colab/results/* ./results/

Mounted at /content/drive


In [ ]:
pip install seaborn

In [ ]:
import seaborn as sns

In [ ]:
pip install  git+https://github.com/ahsank/runml@testv2.01#egg=runml

In [5]:
import runml.pipeline
from runml import pipeline,findata


Todos:



1.   Load previously saved model before training
2.   Allow eval only without test/train spliting
3.   Incremental data load
4.    Add S&P, QQQ etc to the model
5. Save model to google drive






In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
  !cp  /content/drive/MyDrive/colab/results/* ./results/
else:
  ! cp ~/Google\ Drive/My\ Drive/colab/results/* ./results/

Run following in local Jupyter

Run following 2 cells for colab to import libraries

Run following if the library code is modified locally

In [6]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAllocHL(df, stop_loss, profit_factor=1):
    #df['Gain_f'] = (2*df['Gain']+df['Gain_l']+df['Gain_h'])/4
    df['Gain_f'] = df[['Gain','Gain_l']].max(axis=1)
    df['Gain_f'] = df[['Gain_h', 'Gain_f']].min(axis=1)
    df['Alloc'] = df['Accu']/stop_loss - (1-df['Accu'])*profit_factor/abs(df['Gain_f'])
    df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
    return df

In [ ]:
!git clone https://github.com/ahsank/runml

In [ ]:
import os
import sys
path = os.path.expanduser('~/src/runml')
print(path)
if not path in sys.path:
  sys.path.append(path)

from runml import pipeline,findata

/Users/ahsank/src/runml


In [ ]:
import importlib
import sys
path = 'runml'
if not path in sys.path:
  sys.path.append(path)

from runml import pipeline,findata


In [14]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
model = "gstock-8b-test"
tickers = [ 'AEXAF', 'ALSMY', 'AMSSY', 'ASML', 'ADDDF', 'ADYEY',
            'BAK', 'BASFY', 'BESIY', 'COTY', 'COVTY', 'CDMGF', 'CPG', 'CX',
            'DBOEY', 'DPSGY',
            'EADSF', 'EIFZF', 'EDNMF', 'ENGGY', 'EQNR', 'ERIC', 'ERJ','ESEA',
            'FANUY', 'FQVLF', 'GLOB', 'HCMLY', 'HDELY',
            'HENKY', 'HYMTF', 'HMC', 'KNNGF', 'MELI', 'MGA',
            'OZON', 'RACE', 'LYB', 'LVMUY',
            'MYTE', 'NHYDY', 'NOK', 'NU', 'NXPI', 'ONON', 'OTGLY', 'PAX', 'POAHF', 'PROSF',
            'RNMBF', 'RYCEF', 'SAFRF', 'SFTBY',
            'ORAN', 'SAP', 'SE', 'SIEGY', 'STLA',
            'TEF', 'TELNF', 'THQQF', 'TKAMY', 'TRYIY', 'TSM',
            'VALE', 'VNT', 'VWAPY', 'YARIY', 'ZURVY']


In [ ]:
findata.EPOCHS=200
model = "val-6b-test"
tickers = ['AAL', 'ALB', 'APO', 'AQN', 'ARCH', 'ARE',
            'BAH', 'BAX', 'BGFV', 'BXC',
            'CBT', 'CC', 'CCJ', 'CF', 'CHK', 'CI', 'CIVI',
            'CG',  'CMI', 'COKE', 'COP', 'CSL',
            'DAL', 'DD', 'DVN', 'EMN',
            'F', 'FCX', 'FIS', 'FLO', 'FMS', 'GLW', 'GWW',
            'HSII', 'HPE', 'IP', 'IPI', 'IVZ', 'JWN', 'KD', 'KHC', 'KMB',
             'M', 'MMM', 'NFG', 'NTR', 'OMC', 'OLN', 'OSK',
            'PARA', 'PCAR', 'PETS', 'PNR', 'PSTL', 'PXD', 'RCL', 'RH',
            'SAFE', 'SCCO', 'SPTN', 'SPG', 'SWK',
            'T', 'TCN', 'TGT', 'TDG', 'TWI', 'UAL', 'UBA', 'URI',
            'VRTX', 'VZ', 'WDC', 'WHR', 'WSM', 'XOM', 'YORW']

In [ ]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAlloc(df, stop_loss, profit_factor=2):
  df['Alloc'] = df['Accu']/stop_loss - (1-df['Accu'])*profit_factor/abs(df['Gain'])
  df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
  return df

In [ ]:
findata.EPOCHS=200
model = "cry-6b-test"
tickers = [ 'ADA-USD', 'BTC-USD', 'DOT-USD', 'DOGE-USD',
            'ETH-USD', 'LTC-USD', 'MATIC-USD', 'XRP-USD']

In [ ]:
findata.EPOCHS=200
model = "etf-7a-test"
tickers = ['ARKF', 'ARKK', 'ARKW', 'DAPP', 'DIA', 'DTEC', 'EEM', 'FPX',
            'ICLN', 'IJR', 'IPO', 'IXC', 'IXN', 'IXP', 'IWM', 'IYZ',
            'JETS', 'MGK', 'MGV', 'MTUM',
            'ONLN', 'QQQ', 'SMOG', 'SPY', 'TDIV',
            'VNQ', 'VT', 'VTI', 'WDIV', 'XITK',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME', 'XNTK', 'XSW' ]

In [ ]:
from importlib import reload
reload(findata)
reload(pipeline)

<module 'runml.pipeline' from '/Users/ahsank/src/runml/runml/pipeline.py'>

In [7]:
findata.EPOCHS=200
model = "mcap-7a-test"
tickers = ['AAPL', 'ACN', 'ADBE', 'AMD', 'AMZN', 'AVGO', 'BA', 'BKNG', 'BRK-B',
            'CAT', 'CDNS', 'CRM', 'COST', 'CSCO', 'DE', 'DHR', 'DIS',
            'GOOGL', 'FDX', 'HD', 'INTC', 'IBM', 'ISRG',
            'META', 'MMC', 'MSFT', 'NKE', 'NVDA', 'NOW',
            'ORCL', 'PYPL', 'QCOM', 'SBUX', 'TMO', 'TSLA', 'TXN', 'UNH', 'UPS']

In [ ]:
lossfn = "huber_loss"
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(),
                         pipeline.AddVWap(),
                         pipeline.AddMA(200),
                         pipeline.Adj()
                         ]))


#df  = pipeline.runModelCombined(tickers, 'ipos2-3a', mod, False, loss=lossfn, output='low', trading=pipeline.LowTrading)[0]
#df  = pipeline.runModelCombined(tickers, 'ipos2-3a', mod, False, loss=lossfn, output='high', trading=pipeline.HighTrading)[0]
df  = pipeline.runModelCombinedVola(tickers, model, mod, False, loss=lossfn)


In [9]:
addAllocHL(df, 0.10, 1)
df.iloc[(df.Alloc * abs(df.Gain_f)).sort_values(ascending=False).index].head(50)
#df.sort_values('Alloc', ascending=False).head(50)


,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Gain_f,Alloc
34,TSLA,20.29,0.90,2898.95,1607.78,215.49,262.339996,0.22,0.73,1751.47,1387.21,256.929993,0.19,0.79,2927.04,1764.07,224.509995,0.04,0.19,8.473684
23,META,92.89,0.94,1800.88,1271.90,283.25,321.089996,0.13,0.76,1781.77,1040.87,324.200012,0.14,0.65,1602.34,1187.20,291.940002,0.03,0.13,8.938462
31,QCOM,59.17,0.93,774.39,790.32,110.10,121.529999,0.10,0.69,679.15,699.26,121.919998,0.11,0.73,786.88,799.71,107.459999,-0.02,0.10,8.600000
30,PYPL,62.01,0.87,1141.72,1349.76,59.42,66.070000,0.11,0.80,1120.20,994.02,68.470001,0.15,0.65,852.21,794.27,60.900002,0.02,0.11,7.518182
28,NOW,228.77,0.91,3828.98,2613.81,541.75,617.200012,0.14,0.80,3657.19,2599.12,590.270020,0.09,0.70,2948.67,2890.93,552.969971,0.02,0.09,8.100000
11,CRM,127.24,0.96,1486.26,1309.90,204.90,220.949997,0.08,0.78,1144.69,946.79,221.529999,0.08,0.70,1173.31,1279.27,198.050003,-0.03,0.08,9.100000
17,GOOGL,54.01,0.94,619.22,415.23,127.46,141.699997,0.11,0.71,509.79,400.08,137.979996,0.08,0.64,438.87,275.10,130.669998,0.03,0.08,8.650000
25,MSFT,133.91,0.89,1511.52,889.32,316.48,340.790009,0.08,0.66,770.32,404.51,344.859985,0.09,0.63,945.15,643.68,291.730011,-0.08,0.08,7.525000
14,DE,111.27,0.93,2389.83,1087.44,396.78,424.140015,0.07,0.64,1233.72,636.85,428.390015,0.08,0.72,1913.57,1558.70,399.299988,0.01,0.07,8.300000
22,ISRG,132.25,0.82,1752.75,1057.54,285.95,324.250000,0.13,0.74,1592.98,736.30,309.190002,0.08,0.68,1195.99,878.41,297.519989,0.04,0.08,5.950000


In [ ]:
findata.EPOCHS=200
model = "vols-8a-test"
pending = ['ASTR']
tickers = [ 'AIRI', 'APRN', 'AUGX', 'AUR', 'AMSC',  'ATOM',
            'BKKT', 'BGSF', 'BLNK', 'BSGM',
            'CARS', 'CLOV', 'CZOO',
            'DLO', 'DMTK', 'DPSI', 'EVGO', 'EXPR', 'FSLY', 'FTCH',
            'GOVX', 'GTE', 'HIMS', 'HIVE',
            'INLX',  'JOBY',  'KIND', 'KULR',
            'LAZR', 'MMMB', 'MTTR',
            'NEPH', 'NSTG', 'OMQS', 'ONDS', 'OPEN', 'OTLY', 'OUST',
            'RAD', 'REAL', 'RIDE', 'RIG', 'RVYL', 'SFIX', 'STEM',
            'SFT',  'TELL', 'YEXT']
  future = ['WKME', 'CXM']

In [ ]:
pending = ['PAR']
findata.EPOCHS=200
model = "ipos-8a-test"
tickers = ['ABNB', 'ACLS' ,'AI', 'AMBA', 'APP',
           'BILL', 'BMBL', 'CELH', 'CFLT', 'CHGG', 'CRCT', 'CRWD',
           'DASH', 'DBX', 'DDOG', 'DOCN', 'DOCS', 'DOCU', 'DXCM',
           'ESTC', 'ETSY', 'EXPE', 'FOUR', 'GFS', 'GLBE', 'GOGO',
           'INDI', 'INMD', 'INTA', 'IOT','JKS', 'LUMN', 'LYFT',
           'MBLY', 'MDB',  'MNDY', 'MNST', 'MPWR',  'MXL',
           'MTCH',  'NVCR','OKTA', 'OLED', 'OSTK',
           'PERI', 'PANW', 'PAYO', 'PD', 'PLUG', 'PI', 'PINS', 'PTON', 'PUBM',
           'RBLX', 'SNAP', 'SNOW', 'SPLK', 'SPOT','STEM',
           'TEAM', 'TDOC', 'TOST', 'TTD', 'TWLO',
           'U','UI', 'UBER', 'UPWK', 'WOLF', 'VEEV', 'Z', 'ZM', 'ZS']

In [ ]:
findata.EPOCHS=200
model = "cetf-7b-test"
tickers = ['ARGT', 'EWD', 'CUBS', 'FLN', 'FM',
            'ECH', 'EWA', 'EWI', 'EWC', 'EWM', 'EWP', 'EWW', 'EWY', 'EWZ',
            'IDX', 'ILF', 'KEMQ', 'THD', 'TUR', 'XCEM']

In [ ]:
findata.EPOCHS=200
model = "com-7b-test"
tickers = [ 'GLD', 'ISHG', 'PDBC', 'SLV', 'SOYB',
            'TIP', 'TLT', 'UUP', 'VTIP', 'WEAT']

In [ ]:
lossfn = "huber_loss"
pipeline.IS_VERBOSE = False
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(),
                         pipeline.AddVWap(),
                         pipeline.AddMA(200),
                         pipeline.Adj()
                         ]))


#df  = pipeline.runModelCombined(tickers, 'ipos2-3a', mod, False, loss=lossfn, output='low', trading=pipeline.LowTrading)[0]
#df  = pipeline.runModelCombined(tickers, 'ipos2-3a', mod, False, loss=lossfn, output='high', trading=pipeline.HighTrading)[0]
df  = pipeline.runModelCombinedVola(tickers, model, mod, True, loss=lossfn)

In [ ]:
addAllocHL(df, 0.10, 1)
df.iloc[(df.Alloc * abs(df.Gain_f)).sort_values(ascending=False).index].head(50)
#df.sort_values('Alloc', ascending=False).head(50)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Gain_f,Alloc
13,CZOO,4.50,0.74,264.19,830.74,1.22,3.64,1.98,0.40,-0.26,274.09,9.410000,6.72,0.55,71.60,565.95,2.070000,0.69,1.98,7.268687
46,SFT,3.49,0.76,291.59,587.05,1.51,2.90,0.92,0.50,415.67,623.32,5.680000,2.76,0.59,133.85,515.65,1.810000,0.20,0.92,7.339130
5,ASTR,0.69,0.75,53.04,81.70,0.31,0.52,0.68,0.46,50.63,44.71,0.930000,1.99,0.63,16.97,101.92,0.370000,0.20,0.68,7.132353
33,NSTG,4.42,0.90,333.93,534.33,3.07,4.44,0.45,0.68,245.28,285.45,4.870000,0.59,0.68,236.25,359.63,3.370000,0.10,0.45,8.777778
39,RAD,2.06,0.88,159.04,149.70,2.31,3.33,0.44,0.60,104.37,126.53,7.730000,2.35,0.76,118.80,177.94,2.190000,-0.05,0.44,8.527273
35,ONDS,1.03,0.90,30.87,69.62,1.14,1.62,0.42,0.58,5.67,43.22,1.550000,0.36,0.75,24.21,51.17,1.130000,-0.01,0.36,8.722222
40,REAL,1.50,0.91,125.04,203.13,2.61,3.70,0.42,0.62,79.55,138.04,3.520000,0.35,0.78,100.95,136.50,2.320000,-0.11,0.35,8.842857
4,AMSC,3.74,0.86,136.39,120.58,8.77,11.65,0.33,0.55,103.84,95.63,12.820000,0.46,0.71,108.08,106.23,9.450000,0.08,0.33,8.175758
36,OPEN,1.52,0.88,108.76,151.60,3.43,4.39,0.28,0.60,143.01,152.75,4.340000,0.27,0.72,81.32,110.85,3.120000,-0.09,0.27,8.355556
37,OTLY,1.61,0.77,6.27,50.29,1.45,1.90,0.31,0.37,-22.93,21.64,2.560000,0.76,0.74,18.97,64.99,1.490000,0.03,0.31,6.958065


In [ ]:
import pandas as pd
df1 = pipeline.fetch_data('ABNB')
#df1.sort_index(inplace=True)
df1 = df1.round(3)

df2 = pd.read_csv('data/ABNB_2023-04-15.csv', index_col=0)
df2.index =  pd.to_datetime(df2.index)
# df2 = df2.round(3)
#data2 = data.tail(1000)
#pd.concat([df1,df2]).drop_duplicates(keep=False)
#df1.compare(df2)
#df2.index.dtype
df1

In [ ]:
display(df.sort_values('Gain', ascending=False))
display(df[["Buy", "Sell", "Total"]].sum()/len(df))


In [ ]:
results['ABNB'].pdata.__dict__.keys()

In [ ]:
findata.EPOCHS=10
tickers1 = ['ARKK', 'ARKW', 'DAPP', 'DTEC', 'EEM', 'FPX',
            'GBTC', 'GLD',
            'ICLN', 'IJR', 'IPO', 'IPOS', 'IWM',
            'JETS', 'KEMQ',
            'MGK', 'MGV', 'MOAT', 'MTUM',
            'QQQ', 'SLV','SMH', 'SMOG',
            'SPY', 'VNQ', 'VT', 'VTI', 'WDIV',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME' ]
mod = pipeline.RateReturnOnly(pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap()]))
df1 = pipeline.runModelCombined(tickers1, 'etf2b', mod, True)

In [ ]:
df1.sort_values('Gain', ascending=False)

In [ ]:
findata.EPOCHS=200
tickers3 = [
           'AHT', 'AMSC', 'ASTR', 'ATOM',
           'BKKT', 'BGFV', 'BGSF', 'CBNT', 'CLOV',
           'DNMR', 'ERJ', 'EVGO',
           'FSLY',  'FTCH', 'GOGO', 'HIVE',
           'ILAL', 'INLX', 'JMIA', 'JOBY',  'KULR', 'MTTR',
           'MYTE', 'NEPH', 'ONDS', 'MQ',
           'PETS', 'PTON', 'SFIX', 'SFT', 'STNE',
           'ULH', 'VRAR', 'WISH']
lossfn = "huber_loss"
# lossfn = "mean_squared_error"
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df3, results = pipeline.runModelCombined(tickers3, 'vols', mod, False, loss=lossfn)

In [ ]:
df3

In [ ]:
import pandas as pd
seq = list()
for ticker, result in results.items():
    tempdf = result.final_df
    tempdf['predicted_rate'] = (tempdf['adjclose_15']-tempdf['adjclose'])/tempdf['adjclose']
    tempdf['true_rate'] = (tempdf['true_adjclose_15']-tempdf['adjclose'])/tempdf['adjclose']
    seq.append(tempdf[['ticker', 'date', "predicted_rate", 'true_rate']])

detailstat = pd.concat(seq)



In [ ]:
normalstat = detailstat[detailstat['predicted_rate'].between(-2,2) & detailstat['true_rate'].between(-2,2)]
largestat = normalstat[~ normalstat['predicted_rate'].between(-0.25,0.25) & ~ normalstat['true_rate'].between(-0.25,0.25)]
largestat.sample(10)

In [ ]:
ax = largestat.plot.scatter(x='predicted_rate', y='true_rate',c='DarkBlue', figsize=(10,10))
# ax.set_yscale('log')
# ax.set_xscale('log')

In [ ]:
# import matplotlib.pyplot as plt

# plt.hist([normalstat['predicted_rate'],normalstat['predicted_rate']],
#          bins=100, range=(-1,1), color = ['r','g'])

corr = largestat[["predicted_rate", "true_rate"]].corr()

corr.style.background_gradient(cmap='coolwarm')

In [ ]:

sns.regplot(x=largestat["predicted_rate"], y=largestat["true_rate"])


In [ ]:
sns.heatmap(normalstat[["predicted_rate", "true_rate"]].corr(), annot = True, fmt='.2g',cmap= 'coolwarm')

In [ ]:
!rsync -av --ignore-existing results/* /content/drive/MyDrive/colab/results/
